In [11]:
import numpy as np
import geopandas as gpd
import folium
import warnings

from shapely.prepared import prep
from shapely.geometry import Point,Polygon
from shapely.ops import cascaded_union
from scipy import spatial

warnings.filterwarnings('ignore')


In [12]:
# read and filter relevant shapefiles
file_path = '../data/district_shapefiles/District_Boundary.shp'
districts_gpd = gpd.read_file(file_path)
khi_districts_list = ['KARACHI SOUTH','KARACHI EAST','CLIFTON CANTONMENT']
khi_districts_gpd = districts_gpd[districts_gpd.DISTRICT.isin(khi_districts_list)]
khi_districts_gpd = khi_districts_gpd.reset_index(drop=True)

In [13]:
khi_shape = cascaded_union(khi_districts_gpd.geometry)
loc = list(khi_districts_gpd.geometry[0].exterior.coords)[0]
loc = loc[::-1]
map_1 =  folium.Map(location=loc,zoom_start=12)
map_1.add_child(folium.GeoJson(data=khi_shape))
map_1

In [14]:
# changing projection form WGS84 to Web Mercator for caclulating distance
# between mesh points in meters not radians.

poly = gpd.GeoDataFrame({'geometry':[khi_shape]},crs=4326)\
.to_crs(epsg=3857)['geometry'].values[0]

#get the bounding points of the selected polygon
latmin, lonmin, latmax, lonmax = poly.bounds
prep_polygon = prep(poly)
valid_points = []
points = []

#each point in mesh will be 500 meters apart
resolution = 500

# create mesh 
for lat in np.arange(latmin, latmax, resolution):
    for lon in np.arange(lonmin, lonmax, resolution):
        points.append(Point((round(lat,4), round(lon,4))))

# only keep those points that are within the the selected polygon
valid_points.extend(filter(prep_polygon.contains,points))

# change projection back to WGS84 convert list of points to Geodataframe
mesh_gpd = gpd.GeoDataFrame({'geometry':valid_points},crs=3857).to_crs(epsg=4326)
mesh_gpd = mesh_gpd.reset_index()

In [15]:
map_2 =  folium.Map(location=loc,zoom_start=12)
for ind, row in mesh_gpd.iterrows():
    point = row.geometry
    lat, lon = point.y, point.x
    folium.CircleMarker([lat, lon], radius=0.1, color='blue', fill=True, fill_color='blue').add_to(map_2)

map_2

In [16]:
# Getting x and y coordinates from geometry object and using it to generate tesselations.
mesh_gpd['lng_lat'] = mesh_gpd.geometry\
    .apply(lambda coord: (round(coord.x,5),round(coord.y,5)))
vor = spatial.Voronoi(list(mesh_gpd['lng_lat'].values))


# Function I got off some gist from GitHub that converts infinite bordering regions to finite regions. 
# It just works? Apologies to the original author, I am unable to locate the original gist and give due credit.
def func_voronoi_finite_polygons_2d(vor, radius=None):
    """
    Reconstruct infinite voronoi regions in a 2D diagram to finite
    regions.
    Parameters
    ----------
    vor : Voronoi
        Input diagram
    radius : float, optional
        Distance to 'points at infinity'.
    Returns
    -------
    regions : list of tuples
        Indices of vertices in each revised Voronoi regions.
    vertices : list of tuples
        Coordinates for revised Voronoi vertices. Same as coordinates
        of input vertices, with 'points at infinity' appended to the
        end.
    """

    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()*2

    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]

        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue

        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue

            # Compute the missing endpoint of an infinite ridge

            t = vor.points[p2] - vor.points[p1] # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        # sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:,1] - c[1], vs[:,0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)




# Convering infinite border regions to finite regions.
regions,vertices=func_voronoi_finite_polygons_2d(vor,radius=0.07)


#Funciton that clips our resulting polygon within the bounds of our original polygon.
def func_intersect_vor(row):
    return khi_shape.intersection(row.vor)

#Converting regions and vertices to Polygon objects and getting a geodataframe with the final grid.
polys = []
for region in regions:
    poly = vertices[region]
    polys.append(Polygon(poly))

vor_gpd = gpd.GeoDataFrame(polys)
vor_gpd = vor_gpd.rename(columns={
    0:'vor'
})


vor_gpd['bounded_vor'] = vor_gpd.apply(func_intersect_vor,1)
vor_gpd = gpd.GeoDataFrame(vor_gpd,geometry='bounded_vor')
vor_gpd = vor_gpd.drop(columns='vor')
vor_gpd = vor_gpd.reset_index()
vor_gpd = vor_gpd.rename(columns={'index':'vor_id'})
vor_gpd['vor_id'] = vor_gpd.vor_id.apply(lambda x: 'vor'+str(x))

In [17]:
map_3 =  folium.Map(location=loc,zoom_start=12)
for ind, row in vor_gpd.iterrows():
    geometry_obj = row.bounded_vor
    map_3.add_child(folium.GeoJson(geometry_obj))

map_3

In [18]:
map_1.save('../viz/khi_shape_poly.html')
map_2.save('../viz/mesh_poly.html')
map_3.save('../viz/fishnet_grid.html')